Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews


In [2]:
from google.colab import files
files.upload()

Saving database.sqlite.zip to database.sqlite.zip


In [3]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [5]:
!unzip database.sqlite

Archive:  database.sqlite.zip
  inflating: database.sqlite         


In [8]:
conn = sqlite3.connect('database.sqlite')

In [10]:
amznreviews = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", conn) # 3 is nuetral neither + nor -

In [11]:
amznreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525814 entries, 0 to 525813
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      525814 non-null  int64 
 1   ProductId               525814 non-null  object
 2   UserId                  525814 non-null  object
 3   ProfileName             525814 non-null  object
 4   HelpfulnessNumerator    525814 non-null  int64 
 5   HelpfulnessDenominator  525814 non-null  int64 
 6   Score                   525814 non-null  int64 
 7   Time                    525814 non-null  int64 
 8   Summary                 525814 non-null  object
 9   Text                    525814 non-null  object
dtypes: int64(5), object(5)
memory usage: 40.1+ MB


In [12]:
amznreviews.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,525814.000000,525814.000000,525814.000000,525814.000000,5.258140e+05
mean,284599.060038,1.747293,2.209544,4.279148,1.295943e+09
std,163984.038077,7.575819,8.195329,1.316725,4.828129e+07
min,1.000000,0.000000,0.000000,1.000000,9.393408e+08
25%,142730.250000,0.000000,0.000000,4.000000,1.270598e+09
50%,284989.500000,0.000000,1.000000,5.000000,1.310861e+09
75%,426446.750000,2.000000,2.000000,5.000000,1.332634e+09
max,568454.000000,866.000000,878.000000,5.000000,1.351210e+09


In [18]:
#amznreviews100k = amznreviews[0:100000]
#amznreviews100k.to_csv("amazonfoodreviews100k.csv",header=True,index=False)

In [27]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = amznreviews['Score']
positiveNegative = actualScore.map(lambda x: 'positive' if x<3 else 'negative') 
amznreviews['Score'] = positiveNegative

In [29]:
amznreviews.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,negative,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,positive,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,negative,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [38]:
amznreviews = amznreviews.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
deduped_amznreviews = amznreviews.drop_duplicates(subset={'UserId','ProfileName','Score','Time'}, keep='first',inplace=False)

In [39]:
print("fraction of data retained = {}".format(deduped_amznreviews.size/amznreviews.size))

fraction of data retained = 0.6333094972747016


In [40]:
deduped_amznreviews.Score.value_counts()

negative    277627
positive     55376
Name: Score, dtype: int64

**Preprocessing:**

In [46]:
import re
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [48]:
stop = stopwords.words('english')

In [49]:

sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

In [51]:
def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [53]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in deduped_amznreviews['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (deduped_amznreviews['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(deduped_amznreviews['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    
    final_string.append(str1)
    i+=1

In [59]:
len(final_string)

333003

In [76]:
amznreviews_final=deduped_amznreviews.copy()

In [77]:
amznreviews_final['cleansed']= final_string

In [79]:
amznreviews_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333003 entries, 138706 to 302474
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      333003 non-null  int64 
 1   ProductId               333003 non-null  object
 2   UserId                  333003 non-null  object
 3   ProfileName             333003 non-null  object
 4   HelpfulnessNumerator    333003 non-null  int64 
 5   HelpfulnessDenominator  333003 non-null  int64 
 6   Score                   333003 non-null  object
 7   Time                    333003 non-null  int64 
 8   Summary                 333003 non-null  object
 9   Text                    333003 non-null  object
 10  cleansed                333003 non-null  object
dtypes: int64(4), object(7)
memory usage: 30.5+ MB


# BoW

**Unigrams**

In [81]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(amznreviews_final['cleansed'].values)

In [82]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(20))
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))

Most Common Positive Words :  [(b'tast', 33460), (b'like', 31212), (b'product', 27142), (b'one', 19919), (b'flavor', 18795), (b'would', 17357), (b'tri', 17149), (b'use', 14736), (b'good', 14524), (b'coffe', 14414), (b'get', 13375), (b'buy', 13279), (b'order', 12512), (b'food', 11820), (b'dont', 11477), (b'tea', 11312), (b'even', 10691), (b'box', 10567), (b'amazon', 9803), (b'time', 9532)]
Most Common Negative Words :  [(b'like', 124211), (b'tast', 115898), (b'good', 99995), (b'love', 96515), (b'flavor', 96373), (b'use', 93682), (b'great', 92825), (b'one', 87010), (b'product', 82610), (b'tri', 79096), (b'tea', 74751), (b'coffe', 72533), (b'make', 67175), (b'get', 65325), (b'food', 58311), (b'would', 50358), (b'time', 50167), (b'buy', 49041), (b'realli', 47078), (b'eat', 46340)]


**Bi-grams**

In [83]:
count_vect = CountVectorizer(ngram_range=(1,2) )
final_bigram_counts = count_vect.fit_transform(amznreviews_final['cleansed'].values)